In [196]:
##modules
#%matplotlib widget
#%matplotlib inline
#

# matplotlib qt
import mne
import numpy as np
import matplotlib
# Establecer un backend interactivo, como 'Qt5Agg', 'GTK3Agg', etc.
# Esto depende de los backends disponibles en tu sistema.

import matplotlib.pyplot as plt

matplotlib.use('qtagg')  # Asegúrate de que este backend está instalado.

import pandas as pd 
import os
import sys

from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs

from os.path import join as pathjoin
from time import time

from pathlib import Path

from autoreject import AutoReject

# aplicar la acf EN epochs

from statsmodels.tsa.stattools import acf
import numpy as np
import ast

import seaborn as sns



In [2]:
# Variables path
layer_script = "block"

#process=""
disco="g"

# subj = "S01"

#subj = sys.argv[1] ## name of participant list

# Carpeta general
datadir = Path(f"{disco}:\\PROYECTO_SELF")

# #carpetas generales de datos
brain_vision_dir = datadir /"Data_self"/"Export_acw_Brain_Vision"
fif_data = datadir /"Data_self"/ "fif_data" 


#carpeta analysis
output_analysis= datadir / "output_analysis"

acw_analysis_path = output_analysis / "acw"
acw_analysis_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de preprocesado
# output_preproc = datadir / "output_preproc"

# preproc_path = output_preproc / f"preproc_{layer_script}"
# preproc_path.mkdir(parents=True, exist_ok=True)
 
# # Carpeta de epocas "sucias"
# epochs_path = preproc_path / f"epochs_{layer_script}"
# epochs_path.mkdir(parents=True, exist_ok=True) 

# # Carpeta de ICA
# ICA_path = preproc_path / f"ICA_{layer_script}"
# ICA_path.mkdir(parents=True, exist_ok=True)

# # Épocas limpias
# epochs_clean_path = preproc_path / f"epochs_clean_{layer_script}"
# epochs_clean_path.mkdir(parents=True, exist_ok=True)

# #epocas epochs
# epochs_path = Path(preproc_path) / f"epochs_{layer_script}"
# epochs_path.mkdir(parents=True, exist_ok=True)
 

# # Definir la carpeta de output_source antes de usarla
# output_source = Path(r"g:\MOUS_204\output_source")

# source_path = output_source / f"source_{layer_script}"
# source_path.mkdir(parents=True, exist_ok=True)

# #raw_hsp es el raw con fiducials cargados
# raw_hsp_path = source_path / f"raw_hsp"
# raw_hsp_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de forward solution
# fwd_path = source_path / f"fwd"
# fwd_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de inverse solution
# inverse_path = source_path / f"inverse"
# inverse_path.mkdir(parents=True, exist_ok=True)


# mne.utils.set_config('SUBJECTS_DIR', r'\\wsl$\Ubuntu-20.04\usr\local\freesurfer\subjects', set_env=True)


In [3]:

#epochs
combinaciones = ["S1 - S4","S1 - S5", "S2 - S4", "S2 - S5", "S3 - S4", "S3 - S5"]

# combinacion= combinaciones[1]

# Obtener todos los archivos y carpetas dentro de la ruta
subjects = list(fif_data.iterdir())

# Filtrar solo los nombres de sujetos (removiendo "_epochs")
subject_names = [subject.stem.replace("_epochs", "") for subject in subjects]

In [85]:
##import data

autocorrelation_subjects_all= pd.read_pickle(acw_analysis_path / "autocorrelation_subjects_all.pickle")


In [86]:
autocorrelation_subjects_all
df=autocorrelation_subjects_all

In [114]:
df

,Subject,Combination,Epoch,acf_elect_all_epoch_all,acf_mean_elect_all_epoch_all,acw_50_elect_all_epoch_all,acw_0_elect_all_epoch_all,acw_50_elect_mean_epoch_all,acw_0_elect_mean_epoch_all,acw_50_elect_all_epoch_mean,acw_0_elect_all_epoch_mean
0,S01,S1 - S4,1,"[[1.0, 0.9641521820526355, 0.8819571317128331,...","[0.00010869565217391305, 0.0001086956521739099...","[0.02, 0.02, 0.134, 0.02, 0.026, 0.038, 0.038,...","[0.676, 0.69, 0.926, 0.442, 0.446, 0.834, 0.82...","[0.04569491525423727, 0.023457627118644065, 0....","[0.7483389830508476, 0.40535593220338983, 0.59...","[0.07863157894736843, 0.07042105263157897, 0.1...","[1.3477894736842104, 1.3209473684210529, 1.338..."
1,S01,S1 - S4,2,"[[1.0, 0.9565461495433694, 0.8555428500964674,...","[0.00010869565217391305, 0.0001086956521739101...","[0.016, 0.016, 0.024, 0.034, 0.028, 0.034, 0.0...","[0.69, 0.686, 0.826, 1.078, 0.838, 0.82, 0.53,...","[0.04569491525423727, 0.023457627118644065, 0....","[0.7483389830508476, 0.40535593220338983, 0.59...","[0.07863157894736843, 0.07042105263157897, 0.1...","[1.3477894736842104, 1.3209473684210529, 1.338..."
2,S01,S1 - S4,3,"[[1.0, 0.9784079139950476, 0.9295450177303343,...","[0.00010869565217391034, 0.0001086956521739138...","[0.056, 0.034, 0.036, 0.056, 0.056, 0.052, 0.0...","[0.948, 0.604, 0.598, 0.872, 0.856, 0.54, 0.84...","[0.04569491525423727, 0.023457627118644065, 0....","[0.7483389830508476, 0.40535593220338983, 0.59...","[0.07863157894736843, 0.07042105263157897, 0.1...","[1.3477894736842104, 1.3209473684210529, 1.338..."
3,S01,S1 - S4,4,"[[1.0, 0.9897616368251712, 0.9658767930814897,...","[0.00010869565217391923, 0.0001086956521739130...","[0.23, 0.388, 0.382, 0.056, 0.042, 0.376, 0.32...","[2.17, 2.504, 2.132, 1.874, 1.616, 2.158, 1.95...","[0.04569491525423727, 0.023457627118644065, 0....","[0.7483389830508476, 0.40535593220338983, 0.59...","[0.07863157894736843, 0.07042105263157897, 0.1...","[1.3477894736842104, 1.3209473684210529, 1.338..."
4,S01,S1 - S4,5,"[[1.0, 0.9871261837539649, 0.9562252234151949,...","[0.00010869565217391305, 0.0001086956521739192...","[0.04, 0.042, 0.042, 0.034, 0.03, 0.044, 0.04,...","[1.05, 1.218, 1.056, 0.876, 1.042, 1.21, 1.218...","[0.04569491525423727, 0.023457627118644065, 0....","[0.7483389830508476, 0.40535593220338983, 0.59...","[0.07863157894736843, 0.07042105263157897, 0.1...","[1.3477894736842104, 1.3209473684210529, 1.338..."
...,...,...,...,...,...,...,...,...,...,...,...
789,S06,S3 - S5,20,"[[1.0, 0.98669568380137, 0.9539216771308008, 0...","[0.00010869565217390687, 0.0001086956521739130...","[0.108, 0.11, 0.04, 0.098, 0.032, 0.038, 0.05,...","[1.848, 2.128, 0.692, 1.614, 1.124, 1.148, 0.7...","[0.03989830508474576, 0.041661016949152536, 0....","[0.8936610169491527, 1.2153559322033898, 1.066...","[0.08750000000000001, 0.08000000000000002, 0.0...","[1.2695833333333333, 1.3977500000000003, 1.217..."
790,S06,S3 - S5,21,"[[1.0, 0.97917067381707, 0.9293761208011164, 0...","[0.00010869565217390996, 0.0001086956521739099...","[0.056, 0.06, 0.058, 0.058, 0.06, 0.062, 0.076...","[1.048, 1.094, 0.898, 0.864, 1.048, 1.426, 0.5...","[0.03989830508474576, 0.041661016949152536, 0....","[0.8936610169491527, 1.2153559322033898, 1.066...","[0.08750000000000001, 0.08000000000000002, 0.0...","[1.2695833333333333, 1.3977500000000003, 1.217..."
791,S06,S3 - S5,22,"[[1.0, 0.9712133844091575, 0.9016433390384965,...","[0.00010869565217391305, 0.0001086956521739130...","[0.04, 0.038, 0.094, 0.04, 0.032, 0.03, 0.038,...","[2.74, 2.732, 2.554, 2.738, 0.408, 1.926, 2.17...","[0.03989830508474576, 0.041661016949152536, 0....","[0.8936610169491527, 1.2153559322033898, 1.066...","[0.08750000000000001, 0.08000000000000002, 0.0...","[1.2695833333333333, 1.3977500000000003, 1.217..."
792,S06,S3 - S5,23,"[[1.0, 0.9668117679915227, 0.8870714240819707,...","[0.00010869565217391305, 0.0001086956521739130...","[0.038, 0.038, 0.054, 0.266, 0.042, 0.044, 0.1...","[2.838, 2.67, 2.404, 2.916, 2.84, 2.684, 2.528...","[0.0398983050847

In [ ]:
 ## comparaciones ACW_50  entre S1 y S2 y S3

##comparaciñón entre S1 y S2 y S3

# como mi acw estñá calculado en MUCHAS epocas entonces 

## voy a empezar comparando los valores enntre S1 - S4 y S2 - S4

##OBTENER LOS VALORES DE ACW_50 DE S1 - S4 y S2 - S4 en UN SOLO ELECTRODO


##obtener los valores de acw_50 de S1 - S4, cojo de momento el promedio de los valores de acw_50 de todos los electrodos en cada epoca

acw_S1_S4_elect_mean_epoch_all= df[(df["Subject"]=="S01")&(df["Combination"]=="S1 - S4")]["acw_50_elect_mean_epoch_all"]
acw_S2_S4_elect_mean_epoch_all=df[(df["Subject"]=="S01")&(df["Combination"]=="S2 - S4")]["acw_50_elect_mean_epoch_all"].iloc[0]


##aplicar un test de normalidad ssapiro wilk

from scipy.stats import shapiro

print(shapiro(acw_S1_S4_elect_mean_epoch_all))
print(shapiro(acw_S2_S4_elect_mean_epoch_all))


# ##como resultados sale que 
# ShapiroResult(statistic=np.float64(0.9117356446581377), pvalue=np.float64(0.07966628974756569))
# ShapiroResult(statistic=np.float64(0.8207769557430947), pvalue=np.float64(0.003966073269366438))

# por lo tantoo NO SON NORMALES



ShapiroResult(statistic=np.float64(0.9117356446581377), pvalue=np.float64(0.07966628974756569))
ShapiroResult(statistic=np.float64(0.8207769557430947), pvalue=np.float64(0.003966073269366438))


In [ ]:

acw_S1_S4_elect_all_epoch_all= df[(df["Subject"]=="S01")&(df["Combination"]=="S1 - S4")]["acw_50_elect_all_epoch_all"]
acw_S2_S4_elect_all_epoch_all=df[(df["Subject"]=="S01")&(df["Combination"]=="S2 - S4")]["acw_50_elect_all_epoch_all"]

In [ ]:
##PLOTSS

# acw_S1_S4_elect_mean_epoch_all

# acw_S2_S4_elect_mean_epoch_all


In [190]:
acw_S1_S4_elect_all_epoch_all= df[(df["Subject"]=="S01")&(df["Combination"]=="S1 - S4")]["acw_50_elect_all_epoch_all"]
print(type(acw_S1_S4_elect_all_epoch_all), acw_S1_S4_elect_all_epoch_all.shape, len(acw_S1_S4_elect_all_epoch_all[0]))


acw_S2_S4_elect_all_epoch_all= df[(df["Subject"]=="S01")&(df["Combination"]=="S2 - S4")]["acw_50_elect_all_epoch_all"]
print(type(acw_S1_S4_elect_all_epoch_all), acw_S1_S4_elect_all_epoch_all.shape, len(acw_S1_S4_elect_all_epoch_all[0]))

<class 'pandas.core.series.Series'> (19,) 59
<class 'pandas.core.series.Series'> (19,) 59


In [191]:
acw_S1_S4_elect_all_epoch_all_array=np.stack(acw_S1_S4_elect_all_epoch_all)
acw_S2_S4_elect_all_epoch_all_array=np.stack(acw_S2_S4_elect_all_epoch_all)



In [ ]:
plt.plot(acw_S1_S4_elect_all_epoch_all_array)
plt.show()


In [201]:
plt.figure(figsize=(8, 6))
sns.boxplot(data=[acw_S1_S4_elect_all_epoch_all_array.flatten(),
                  acw_S2_S4_elect_all_epoch_all_array.flatten()])
plt.xticks([0, 1], ['Condición 1', 'Condición 2'])
plt.ylabel('Valores de la medida')
plt.title('Comparación de la medida entre condiciones')
plt.show()

In [199]:
plt.figure(figsize=(8, 6))
sns.violinplot(data=[acw_S1_S4_elect_all_epoch_all_array.flatten(),
                     acw_S2_S4_elect_all_epoch_all_array.flatten()])
plt.xticks([0, 1], ['Condición 1', 'Condición 2'])
plt.ylabel('Valores de la medida')
plt.title('Distribución de la medida en ambas condiciones')
plt.show()

In [197]:
plt.figure(figsize=(8, 6))
sns.kdeplot(acw_S1_S4_elect_all_epoch_all_array.flatten(), label="Condición 1", fill=True)
sns.kdeplot(acw_S1_S4_elect_all_epoch_all_array.flatten(), label="Condición 2", fill=True)
plt.xlabel('Valor de la medida')
plt.ylabel('Densidad')
plt.title('Distribución KDE de la medida en ambas condiciones')
plt.legend()
plt.show()



pandas.core.series.Series

In [217]:
comb[0]

'S3 - S5'

In [216]:
for comb in df.groupby("Combination")["acw_50_elect_all_epoch_all"]:
    print("comb", comb)

   

comb ('S1 - S4', 0      [0.02, 0.02, 0.134, 0.02, 0.026, 0.038, 0.038,...
1      [0.016, 0.016, 0.024, 0.034, 0.028, 0.034, 0.0...
2      [0.056, 0.034, 0.036, 0.056, 0.056, 0.052, 0.0...
3      [0.23, 0.388, 0.382, 0.056, 0.042, 0.376, 0.32...
4      [0.04, 0.042, 0.042, 0.034, 0.03, 0.044, 0.04,...
                             ...                        
671    [0.12, 0.182, 0.196, 0.104, 0.11, 0.204, 0.254...
672    [0.018, 0.02, 0.024, 0.018, 0.022, 0.024, 0.02...
673    [0.08, 0.084, 0.096, 0.116, 0.042, 0.058, 0.1,...
674    [0.2, 0.114, 0.184, 0.202, 0.036, 0.036, 0.044...
675    [0.486, 0.396, 0.522, 0.132, 0.1, 0.064, 0.398...
Name: acw_50_elect_all_epoch_all, Length: 133, dtype: object)
comb ('S1 - S5', 19     [0.022, 0.02, 0.018, 0.036, 0.056, 0.034, 0.02...
20     [0.038, 0.026, 0.032, 0.038, 0.074, 0.05, 0.02...
21     [0.032, 0.034, 0.044, 0.044, 0.036, 0.026, 0.0...
22     [0.036, 0.052, 0.054, 0.032, 0.17, 0.05, 0.052...
23     [0.028, 0.03, 0.03, 0.028, 0.034, 0.036, 0

In [218]:
for comb, values in df.groupby("Combination")["acw_50_elect_all_epoch_all"]:
    plt.figure(figsize=(8, 6))
    sns.kdeplot(np.stack(values), label=comb, fill=True)

In [ ]:
acw_50_elect_all_epoch_all = {
    comb: np.stack(values)  # Convertir listas en arrays de NumPy
    for comb, values in df[df["Subject"] == "S01"].groupby("Combination")["acw_50_elect_all_epoch_all"]
}




In [228]:
print(acw_50_elect_all_epoch_all['S1 - S4'].shape)
print(acw_50_elect_all_epoch_all['S1 - S4'].flatten().shape)

(19, 59)
(1121,)


In [255]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

acw_50_elect_all_epoch_all = {
    comb: {
        "data": np.stack(values),  # Convertir listas en arrays de NumPy
        "num_epocas": len(values),  # Guardar el número de épocas
        "num_electrodos": np.stack(values).shape[1],  # Número de electrodos (siempre 59)
        }
    for comb, values in df[df["Subject"] == "S01"].groupby("Combination")["acw_50_elect_all_epoch_all"]
}


In [254]:

# # Verificar el diccionario generado
# for key, value in acw_50_elect_all_epoch_all.items():
#     print(f"Condición: {key} → Shape {value.shape}")  # Debería ser (19, 59) para cada combinación



plt.figure(figsize=(10, 6))

# Extraer los valores a graficar
##flatten pasa todo a una misma dimensiñón, permitiendo graficar todo
data_boxplot = [acw_50_elect_all_epoch_all[comb]["data"].flatten() for comb in acw_50_elect_all_epoch_all.keys()]

# Graficar Boxplot con seaborn
sns.boxplot(data=data_boxplot)

# Etiquetas en el eje X
plt.xticks(range(len(acw_50_elect_all_epoch_all.keys())), acw_50_elect_all_epoch_all.keys(), rotation=15)

plt.ylabel('Valores de la medida')
plt.title('Comparación de la medida entre combinaciones experimentales')

plt.show()


In [232]:
# Obtener la lista única de sujetos en el DataFrame
sujetos_unicos = df["Subject"].unique()

combinaciones_unicas = df["Combination"].unique()
combinaciones_unicas

array(['S1 - S4', 'S1 - S5', 'S2 - S4', 'S2 - S5', 'S3 - S4', 'S3 - S5'],
      dtype=object)

In [237]:
acw_50_elect_all_epoch_all.keys()

dict_keys(['S1 - S4', 'S1 - S5', 'S2 - S4', 'S2 - S5', 'S3 - S4', 'S3 - S5'])

In [239]:
data_boxplot

[array([0.012, 0.016, 0.018, ..., 0.354, 1.068, 0.034], shape=(1357,)),
 array([0.014, 0.014, 0.012, ..., 0.226, 0.792, 0.118], shape=(1416,)),
 array([0.01 , 0.01 , 0.01 , ..., 0.052, 0.298, 0.032], shape=(1357,)),
 array([0.008, 0.008, 0.01 , ..., 0.168, 0.054, 0.04 ], shape=(1416,)),
 array([0.01 , 0.012, 0.018, ..., 0.128, 0.096, 0.05 ], shape=(1357,)),
 array([0.008, 0.008, 0.008, ..., 0.38 , 1.124, 0.442], shape=(1416,))]

In [242]:

# Bucle para iterar sobre cada condicion


# Crear la figura para el boxplot
plt.figure(figsize=(10, 6))

# Extraer los valores a graficar, aplanando los arrays para cada combinación

data_boxplot = [acw_50_elect_all_epoch_all[comb].flatten() for comb in acw_50_elect_all_epoch_all.keys()]


# Graficar Boxplot con seaborn
sns.boxplot(data=data_boxplot)

# Etiquetas en el eje X
plt.xticks(range(len(acw_50_elect_all_epoch_all.keys())), acw_50_elect_all_epoch_all.keys(), rotation=15)

    # **Ajustar el eje Y a un rango de 2 segundos**
plt.ylim(0, 2)  # 📌 Esto fuerza el eje Y entre 0 y 2 segundos

# Etiquetas y título del gráfico
plt.ylabel('Valores de la medida')
plt.title(f'Comparación de la medida entre combinaciones experimentales de todos los sujetos')

# Mostrar el gráfico para el sujeto actual
plt.show()


In [243]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Extraer las combinaciones únicas
combinaciones_unicas = list(acw_50_elect_all_epoch_all.keys())

# Crear un diccionario para almacenar las medias de cada combinación
medias = {comb: np.mean(acw_50_elect_all_epoch_all[comb]) for comb in combinaciones_unicas}

# Convertir en DataFrame para análisis
df_medias = pd.DataFrame(list(medias.items()), columns=['Condición', 'Media'])

# Mostrar las medias ordenadas
df_medias = df_medias.sort_values(by='Media', ascending=False)
print("\n📊 Medias ordenadas de mayor a menor:")
print(df_medias)

# Obtener los datos como listas separadas para cada condición
datos = [acw_50_elect_all_epoch_all[comb].flatten() for comb in combinaciones_unicas]

# 🔹 Prueba de Normalidad (Shapiro-Wilk)
print("\n📊 Prueba de normalidad (Shapiro-Wilk):")
for i, comb in enumerate(combinaciones_unicas):
    stat, p = stats.shapiro(datos[i])
    print(f"  {comb}: W={stat:.4f}, p={p:.4f}")

# 🔹 Elección de la prueba estadística
print("\n📊 Comparación de medias:")
if all(stats.shapiro(data)[1] > 0.05 for data in datos):  # Si todos los p-valores > 0.05, asumimos normalidad
    print("✅ Datos normales → Realizando ANOVA...")
    stat, p_anova = stats.f_oneway(*datos)
    print(f"  ANOVA: F={stat:.4f}, p={p_anova:.4f}")

    # 🔹 Comparaciones post hoc (Tukey HSD si ANOVA es significativo)
    if p_anova < 0.05:
        tukey = pairwise_tukeyhsd(np.concatenate(datos), 
                                  np.repeat(combinaciones_unicas, [len(d) for d in datos]))
        print("\n📊 Comparaciones post hoc (Tukey HSD):")
        print(tukey)

else:
    print("⚠️ Datos NO normales → Realizando Kruskal-Wallis...")
    stat, p_kruskal = stats.kruskal(*datos)
    print(f"  Kruskal-Wallis: H={stat:.4f}, p={p_kruskal:.4f}")

    # 🔹 Comparaciones post hoc (Mann-Whitney si Kruskal es significativo)
    if p_kruskal < 0.05:
        print("\n📊 Comparaciones post hoc (Mann-Whitney U test):")
        for i in range(len(combinaciones_unicas)):
            for j in range(i + 1, len(combinaciones_unicas)):
                u_stat, p_u = stats.mannwhitneyu(datos[i], datos[j])
                print(f"  {combinaciones_unicas[i]} vs {combinaciones_unicas[j]}: U={u_stat:.4f}, p={p_u:.4f}")



📊 Medias ordenadas de mayor a menor:
  Condición     Media
0   S1 - S4  0.098836
4   S3 - S4  0.098037
3   S2 - S5  0.089537
5   S3 - S5  0.083117
2   S2 - S4  0.077718
1   S1 - S5  0.069147

📊 Prueba de normalidad (Shapiro-Wilk):
  S1 - S4: W=0.3967, p=0.0000
  S1 - S5: W=0.4159, p=0.0000
  S2 - S4: W=0.3535, p=0.0000
  S2 - S5: W=0.4546, p=0.0000
  S3 - S4: W=0.4309, p=0.0000
  S3 - S5: W=0.3750, p=0.0000

📊 Comparación de medias:
⚠️ Datos NO normales → Realizando Kruskal-Wallis...
  Kruskal-Wallis: H=113.9804, p=0.0000

📊 Comparaciones post hoc (Mann-Whitney U test):
  S1 - S4 vs S1 - S5: U=1066124.0000, p=0.0000
  S1 - S4 vs S2 - S4: U=1078066.5000, p=0.0000
  S1 - S4 vs S2 - S5: U=980698.5000, p=0.3439
  S1 - S4 vs S3 - S4: U=900654.0000, p=0.3253
  S1 - S4 vs S3 - S5: U=1050585.5000, p=0.0000
  S1 - S5 vs S2 - S4: U=1023144.0000, p=0.0031
  S1 - S5 vs S2 - S5: U=912722.5000, p=0.0000
  S1 - S5 vs S3 - S4: U=832637.0000, p=0.0000
  S1 - S5 vs S3 - S5: U=989402.5000, p=0.5462
  S2

In [251]:
acw_50_elect_all_epoch_all.keys()

dict_keys(['S1 - S4', 'S1 - S5', 'S2 - S4', 'S2 - S5', 'S3 - S4', 'S3 - S5'])

In [250]:
acw_50_elect_all_epoch_all["S1 - S4"]

acw_50_elect_all_epoch_all["S1 - S4"][:,0]

array([0.012, 0.014, 0.018, 0.018, 0.02 , 0.018, 0.02 , 0.03 , 0.14 ,
       0.028, 0.028, 0.06 , 0.156, 0.058, 0.036, 0.056, 0.436, 0.172,
       0.12 , 0.018, 0.08 , 0.2  , 0.486])

In [245]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.multicomp import pairwise_tukeyhsd

for cond in grupo_s1:
    print(f"Condición: {cond} - Shape: {acw_50_elect_all_epoch_all[cond].shape}")

# Asegurar que los arrays tengan la misma forma antes de calcular la media
grupo_colapsado = {
    "S1": np.mean(np.stack([acw_50_elect_all_epoch_all[cond] for cond in grupo_s1]), axis=0),
    "S2": np.mean(np.stack([acw_50_elect_all_epoch_all[cond] for cond in grupo_s2]), axis=0),
    "S3": np.mean(np.stack([acw_50_elect_all_epoch_all[cond] for cond in grupo_s3]), axis=0),
}

# Extraer las medias
medias_grupo = {grupo: np.mean(grupo_colapsado[grupo]) for grupo in grupo_colapsado.keys()}

# Convertir en DataFrame para visualizar las medias ordenadas
df_medias = pd.DataFrame(list(medias_grupo.items()), columns=['Grupo', 'Media'])
df_medias = df_medias.sort_values(by='Media', ascending=False)

print("\n📊 Medias de los grupos S1, S2 y S3:")
print(df_medias)

# Obtener los datos como listas separadas
datos = [grupo_colapsado[grupo].flatten() for grupo in grupo_colapsado.keys()]

# 🔹 Prueba de Normalidad (Shapiro-Wilk)
print("\n📊 Prueba de normalidad (Shapiro-Wilk):")
for i, grupo in enumerate(grupo_colapsado.keys()):
    stat, p = stats.shapiro(datos[i])
    print(f"  {grupo}: W={stat:.4f}, p={p:.4f}")

# 🔹 Comparación de medias
print("\n📊 Comparación de S1, S2 y S3:")
if all(stats.shapiro(data)[1] > 0.05 for data in datos):  # Si los datos son normales
    print("✅ Datos normales → Realizando ANOVA...")
    stat, p_anova = stats.f_oneway(*datos)
    print(f"  ANOVA: F={stat:.4f}, p={p_anova:.4f}")

    # Comparaciones post hoc (Tukey HSD si ANOVA es significativo)
    if p_anova < 0.05:
        tukey = pairwise_tukeyhsd(np.concatenate(datos), 
                                  np.repeat(["S1", "S2", "S3"], [len(d) for d in datos]))
        print("\n📊 Comparaciones post hoc (Tukey HSD):")
        print(tukey)

else:
    print("⚠️ Datos NO normales → Realizando Kruskal-Wallis...")
    stat, p_kruskal = stats.kruskal(*datos)
    print(f"  Kruskal-Wallis: H={stat:.4f}, p={p_kruskal:.4f}")

    # Comparaciones post hoc (Mann-Whitney si Kruskal es significativo)
    if p_kruskal < 0.05:
        print("\n📊 Comparaciones post hoc (Mann-Whitney U test):")
        for i in range(len(grupo_colapsado.keys())):
            for j in range(i + 1, len(grupo_colapsado.keys())):
                u_stat, p_u = stats.mannwhitneyu(datos[i], datos[j])
                print(f"  {list(grupo_colapsado.keys())[i]} vs {list(grupo_colapsado.keys())[j]}: U={u_stat:.4f}, p={p_u:.4f}")

# 🔹 Graficar el Boxplot
plt.figure(figsize=(8, 6))
sns.boxplot(data=datos)
plt.xticks(range(len(grupo_colapsado.keys())), grupo_colapsado.keys())
plt.ylabel('Valores de la medida')
plt.title('Comparación entre S1, S2 y S3')
plt.ylim(0, 2)  # Ajuste del eje Y

plt.show()


Condición: S1 - S4 - Shape: (23, 59)
Condición: S1 - S5 - Shape: (24, 59)


ValueError: all input arrays must have the same shape

KeyError: False